In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, IntegerType
import re

In [0]:
# Definindo o catálogo e schemas
CATALOGO = "catalogo_energia"
SCHEMA_BRONZE = "bronze"
SCHEMA_SILVER = "silver"

In [0]:
# Lendo as tabelas Delta da camada Bronze
df_geral_bronze = spark.read.table(f"{CATALOGO}.{SCHEMA_BRONZE}.consumo_geral")
df_limpa_bronze = spark.read.table(f"{CATALOGO}.{SCHEMA_BRONZE}.energia_limpa")

print(f"Total de linhas (Geral): {df_geral_bronze.count()}")
print(f"Total de linhas (Limpa): {df_limpa_bronze.count()}")

In [0]:
def limpar_e_padronizar(df):
    
    # 1. Tratamento de Nulos em Colunas Numéricas
    # Substituir nulos por 0 nas colunas que serão usadas em cálculos.
    colunas_numericas = ["consumo_em_kwh", "valor_da_conta", "valor_de_imposto"]
    df = df.fillna(0, subset=colunas_numericas)
    
    # 2. Padronização de Colunas de Texto
    # Garante consistência de agrupamento para a análise.
    colunas_texto_chave = ["regiao", "cidade", "bairro", "tipo_de_cliente"]
    
    for col in colunas_texto_chave:
        # F.trim: remove espaços antes/depois; F.initcap: 'sao paulo' -> 'Sao Paulo'.
        df = df.withColumn(col, F.initcap(F.trim(F.col(col))))

    # 3. Conversão de Tipos (Cast) - Garantindo DoubleType para precisão
    for col in colunas_numericas:
        df = df.withColumn(col, F.col(col).cast(DoubleType()))

    # 4. Tratamento e Mapeamento do Mês
    # Converte o nome do mês (string) para o número do mês (integer)
    df = df.withColumn(
        "mes_de_referencia",
        F.date_format(
            F.to_date(F.col("mes_de_referencia").cast("string"), "MMMM"), 
            "M"
        ).cast(IntegerType())
    )

    # 4. Padronização de Nomes de Cliente
    df = df.withColumn("tipo_de_cliente", 
             F.when(F.col("tipo_de_cliente") == "Residential", "Residencial")
             .when(F.col("tipo_de_cliente") == "Commercial", "Comercial")
             .otherwise(F.col("tipo_de_cliente")))
        
    return df

# Aplicando a função em ambos os DataFrames
df_geral_limpo = limpar_e_padronizar(df_geral_bronze)
df_limpa_limpo = limpar_e_padronizar(df_limpa_bronze)

In [0]:
# 1. Adicionar o indicador de tipo de consumo (necessário para a análise comparativa)
df_geral_com_tipo = df_geral_limpo.withColumn("tipo_de_consumo", F.lit("Geral"))
df_limpa_com_tipo = df_limpa_limpo.withColumn("tipo_de_consumo", F.lit("Limpa"))

# 2. Unificar os DataFrames (Union)
# unionByName lida melhor com DataFrames que podem ter colunas em ordens diferentes.
df_unificado_silver = df_geral_com_tipo.unionByName(df_limpa_com_tipo)

print(f"Total de linhas Unificado (Silver): {df_unificado_silver.count()}")

# Checagem: verificar se as contagens estão corretas
df_unificado_silver.groupBy("tipo_de_consumo").count().show()

In [0]:
# US-2.2: Criar a Coluna de Chave de Negócio Única (BK)
# Combinação de localização, período e tipo de cliente.
df_silver_final = df_unificado_silver.withColumn(
    "chave_negocio_bk",
    F.sha2(
        # Concatena os campos chave com um separador '_'
        F.concat_ws("_", 
            F.col("regiao"), 
            F.col("cidade"), 
            F.col("bairro"), 
            F.col("mes_de_referencia"), 
            F.col("tipo_de_cliente"),
            F.col("tipo_de_consumo")
        ), 
        256 # Algoritmo SHA-256
    )
)